In [1]:
import matplotlib.pyplot as mplt
import pandas as pd
import numpy as np
import geopandas as gpd
#import cProfile
#import pstats

from functools import partial
#from tqdm import tqdm

from helper import DataHandler, Plot
from functools import partial

from partition import Partition, GeographicPartition
from markovchain import SingleMetricOptimizer
import constraints
from markovchain import recom

In [2]:
"Load data"
handler = DataHandler()
graph = handler.load_graph()
chicago = handler.load_chicago()
candidates = handler.load_candidates()  # set of nodes
geo_centers = handler.load_geo_centers()
geo_candidates = handler.load_geo_candidates()
#travel_times_walking = handler.load_travel_walking()
#travel_times = handler.load_travel_times() # travel time between blocks is 10 minutes
newtravel = handler.load_newtravel()
geometries = handler.load_geometries()
plt = Plot()

In [3]:
from graph import Graph
testgraph = Graph.from_geodataframe(chicago)

/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/falcomchain/graph/graph.py:266: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


In [3]:
"Initial partition"
pop = sum(graph.nodes[node]['pop'] for node in graph.nodes)
area = sum(graph.nodes[node]['area'] for node in graph.nodes)   
#area 
#avg_density = area / pop

# nodelarin density check i burda yapilip sonuc attribute olarak atansin. Attribute ismi simdilik density

column_names = ['pop', 'area', 'real_phc', 'avg_density'] # avg_density attribute ata. If density is not considered, then we have to pass false.
#check_density = ?

import partition
my_updaters = {
    "population": partition.Tally(column_names[0], alias="population"),
    "cut_edges": partition.cut_edges,
    "perimeter": partition.perimeter,
    "area": partition.Tally("area", alias="area"),}

In [ ]:
initial_solution = GeographicPartition.from_random_assignment(
travel_times=newtravel,
graph=graph,
capacity_level = 1,
epsilon=0.01,
column_names = column_names,
pop_target = pop // 100,
#density = 0.5,
updaters=my_updaters)


In [4]:
from collections import defaultdict
from collections.abc import Mapping
from typing import Dict, Union, Optional, DefaultDict, Set, Type, Tuple
from partition import get_assignment

part_assignment = handler.load_final_assignment()

Assignment = get_assignment(part_assignment, graph, column_names, newtravel)

In [4]:
import pickle
path1 = '/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/falcomchain/prepared_data/initial_assignment.pkl'
path2 = '/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/falcomchain/prepared_data/initial_teams.pkl'
#initial_assignment = dict(initial_solution.assignment)
#initial_teams = initial_solution.teams
#with open(path1, 'wb') as file:
#                pickle.dump(initial_assignment, file)
#with open(path2, 'wb') as file:
#                pickle.dump(initial_teams, file)

def load(path):
    with open(path, 'rb') as file:
        return pickle.load(file)
    
initial_assignment = load(path1)
initial_teams = load(path2)

In [5]:
initial_solution = GeographicPartition(graph=graph, assignment=initial_assignment, teams=initial_teams, travel_times=newtravel, capacity_level=1, updaters=my_updaters)

In [6]:
"Plot initial partition"
#m, regions, chicago, geocenters = initial_solution.plot_map(attr="district")
#m

"Plot side by side"
fig, regions_initial_new, regions_final_new, centers, others = plt.compare(initial_solution, initial_solution)
fig

In [4]:
initial_solution2 = GeographicPartition.from_random_assignment(
travel_times=newtravel,
graph=graph,
capacity_level = 2,
epsilon=0.01,
column_names = column_names,
pop_target = pop // 100,
#density = 0.5,
updaters=my_updaters)

--------------iteration starts
remaining pop 2783891
number of facility valid nodes 14794
number of pop and facility valid nodes 33
only pop valid 0
ideal pop 27838.0
epsilon 0.010000000000000037
root pop 2783891
root facility True
complement facility valid or root 0
plus complement pop vali or root 0
--------------iteration ends
Created district 1
--------------iteration starts
remaining pop 2728428
number of facility valid nodes 13748
number of pop and facility valid nodes 25
only pop valid 0
ideal pop 27891.25
epsilon 0.008071707076592157
root pop 2728428
root facility True
complement facility valid or root 0
plus complement pop vali or root 0
--------------iteration ends
Created district 2
--------------iteration starts
remaining pop 2672527
number of facility valid nodes 13824
number of pop and facility valid nodes 32
only pop valid 0
ideal pop 27835.0
epsilon 0.009893299802407078
root pop 2672527
root facility True
complement facility valid or root 0
plus complement pop vali or r

In [ ]:
initial_solution2.teams

In [16]:
m.save("map0.html")

In [6]:
"Proposal and constraints"   # See how we call this in optimizer. 
proposal = partial(
    recom,
    pop_target = pop // 100,
    column_names = column_names,
    epsilon=0.02,
    density = None)
constraints = constraints.within_percent_of_ideal_population(initial_solution)

In [7]:
"Optimizer"
 
#sum_travel_radius = lambda p: np.sum(p.radius.values())
#num_cut_edges = lambda p: len(p["cut_edges"])
average_radius = lambda p: np.sum(((np.array(list(p.radius.values())) - np.sum(list(p.radius.values())) / len(p))**2))

optimizer = SingleMetricOptimizer(
    proposal=proposal,
    constraints=constraints,
    initial_state=initial_solution,
    optimization_metric=average_radius,
    maximize=False)

In [8]:
"Tilted Runs"

total_steps = 1000
partitions = {}

min_scores_tilt = np.zeros(total_steps)
for i, part in enumerate(optimizer.tilted_run(total_steps, p=0.125, with_progress_bar=True)):
    min_scores_tilt[i] = optimizer.best_score
    partitions[i] = optimizer.best_part
final_partition = optimizer.best_part


  0%|          | 0/1000 [00:00<?, ?it/s]

--------------iteration starts
remaining pop 55607
number of facility valid nodes 174
number of pop and facility valid nodes 2
only pop valid 0
ideal pop 27838.0
epsilon 0.020000000000000073
root pop 55607
root facility True
complement facility valid or root 174
plus complement pop vali or root 1
--------------iteration ends
Created district 1
--------------iteration starts
remaining pop 27528
number of facility valid nodes 93
number of pop and facility valid nodes 1
only pop valid 231
ideal pop 27717.5
epsilon 0.015739514747001065
root pop 27528
root facility True
complement facility valid or root 86
plus complement pop vali or root 1
--------------iteration ends
Created district 2


  0%|          | 1/1000 [00:00<07:34,  2.20it/s]

part 1
flow {'in': {8197, 20487, 10249, 10250, 24590, 24591, 36941, 36947, 36948, 36949, 16571, 8409, 8414, 12535, 12542, 12543, 12544, 12547, 12548, 12549, 26888, 12553, 12552, 12554, 272, 293, 295, 12584, 296, 4394, 4395, 4396, 18738, 37171, 37172, 37173, 37174, 37175, 16696, 37177, 37178, 37179, 37180, 37181, 37176, 16714, 16716, 24923, 16734, 16742, 4474, 18816, 4481, 18818, 24968, 24969, 24970, 24971, 24972, 37264, 37265, 37293, 18864, 18865, 18866, 17997, 27079, 4556, 4557, 14803, 14805, 14806, 12790, 6674, 19018, 14925, 14926, 14927, 14928, 14929, 14930, 19027, 19025, 19026, 29379, 2757, 23241, 2762, 23242, 35560, 37667, 37668, 13593, 37671, 37672, 35623, 35626, 35624, 35628, 35629, 35630, 35631, 35632, 842, 25426, 25450, 25451, 21364, 9077, 9078, 23420, 23429, 17683, 9098, 9099, 9100, 912, 31635, 31636, 31637, 31638, 31639, 17304, 31640, 19354, 31642, 17306, 17307, 17310, 17309, 31644, 31649, 31645, 31646, 31647, 31648, 31650, 31651, 17311, 938, 2998, 2999, 5048, 17694, 7133, 9

ValueError: min() iterable argument is empty

In [ ]:
"Plot side by side"
fig, regions_initial_new, regions_final_new, centers, others = plt.compare(initial_partition, final_partition)
fig


# Last result for average radius with fixed travel time

In [ ]:
import matplotlib.pyplot as mplt
fig, ax = mplt.subplots(figsize=(12,6))
mplt.plot(min_scores_tilt, label="Tilted Run")
mplt.xlabel("Steps", fontsize=20)
mplt.ylabel("Sum of radius_average", fontsize=20)
mplt.legend()
mplt.show()

In [ ]:
"Plot side by side"
fig, regions_initial_new, regions_final_new, centers, others = plt.compare(initial_partition, final_partition)
fig

In [ ]:
#sum_travel_radius = lambda p: np.sum(p.radius.values())
num_cut_edges = lambda p: len(p["cut_edges"])
average_radius = lambda p: np.sum(abs((np.array(list(p.radius.values())) - np.sum(list(p.radius.values())) / len(p))**2))

In [ ]:
import gerrytools

import matplotlib.pyplot as plt
from gerrytools.scoring import *
from gerrytools.plotting import *
import gerrytools.plotting.colors as colors
import numpy as np

N = len(regions_final_new)

dists = regions_final_new.to_crs("EPSG:3857")
dists["final_district"] = dists["final_district"].astype(int)
dists=dists.sort_values(by="final_district")
dists["colorindex"] = list(range(N))
dists["color"] = colors.districtr(N)

ax = drawplan(chicago, assignment="final_district", overlays=None)

In [ ]:
import maup
from maup import repair

In [ ]:
maup.doctor(chicago)

In [ ]:
repair.count_holes(chicago)

In [ ]:
maup.repair.autorepair(chicago)

In [ ]:
final_partition.plot(chicago, figsize=(10, 10), cmap="tab20")
mplt.axis('off')
mplt.show()

In [ ]:
import numpy as np
print("1) Number of Cut Edges")
print("    Best score: ", optimizer.best_score)
print("    Initial score: ", len(initial_partition["cut_edges"]))
print("2) Sum of Travel Radius")
print("    Initial: ", np.sum(list(initial_partition.radius.values())))
print("    Final: ", np.sum(list(final_partition.radius.values())))
print("3) Sum of Radius Devitation")
print("    Initial: ", sum(abs(np.array(list(initial_partition.radius.values())) - sum(x for x in initial_partition.radius.values()) / len(initial_partition))**2))
print("    Final: ", sum(abs(np.array(list(final_partition.radius.values())) - sum(x for x in final_partition.radius.values())/ len(final_partition))**2 ))

In [ ]:
"""pcompress
radiuss = {}
i=0
for optimizer in Record(optimizer.tilted_run(total_steps, p=0.125, with_progress_bar=True), "pa-run.chain"):
    # normal chain stuff here
    partition = optimizer.best_part
    radius = partition.radius
    radiuss[i] = radius 
    i+= 1"""

In [ ]:
"Watch"
%matplotlib inline
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('png')

import pandas as pd

import matplotlib.cm as mcm
import matplotlib.pyplot as mplt
import networkx as nx
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

frames = []
district_data = []

for i, partition in enumerate(recom_chain):
    for district_name in partition.perimeter.keys():
        population = partition.population[district_name]
        perimeter = partition.perimeter[district_name]
        area = partition.area[district_name]
        district_data.append((i, district_name, population, perimeter, area))

    buffer = io.BytesIO()
    fig, ax = plt.subplots(figsize=(10,10))
    partition.plot(ax=ax, cmap='tab20')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image = Image.open(buffer)
    frames.append(image)
    plt.close(fig)

df = pd.DataFrame(
    district_data,
    columns=[
        'step',
        'district_name',
        'population',
        'perimeter',
        'area'
    ]
)

def show_frame(idx):
    clear_output(wait=True)
    display(frames[idx])

slider = widgets.IntSlider(value=0, min=0, max=len(frames)-1, step=1, description='Frame:')
slider.layout.width = '500px'
widgets.interactive(show_frame, idx=slider)
#df.head(5)
#The perimeter and area attributes are actually not present in the MN_precincts.geojson file, but the GeographicPartition class will calculate them at instantiation time using the geometries provided in the file.

# Save

In [ ]:
#final_partition = optimizer.best_part
#for partition in pcompress.Record(optimizer, "run.chain", executable="pv", extreme=True):
#    print(partition.population)

#final_partition = optimizer.best_part
#handler = DataHandler()
#handler.load_final_assignment()
#final_partition = optimizer.best_part
#final_assignment = dict(final_partition.assignment)
#pd.to_pickle(final_assignment, '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/final_assignment.pkl')
#final_assignment = handler.load_final_assignment()
#final_assignment = Partition(graph, final_assignment, updaters=my_updaters)

# Local Search

In [ ]:
# We can run each of the optimization methods and collect some data

total_steps = 10000

# Short Bursts
min_scores_sb = np.zeros(total_steps)
for i, part in enumerate(optimizer.short_bursts(5, 2000, with_progress_bar=True)):
    min_scores_sb[i] = optimizer.best_score

# Simulated Annealing
min_scores_anneal = np.zeros(total_steps)
for i, part in enumerate(
    optimizer.simulated_annealing(
        total_steps,
        optimizer.jumpcycle_beta_function(200, 800),
        beta_magnitude=1,
        with_progress_bar=True
    )
):
    min_scores_anneal[i] = optimizer.best_score